# Chapter 4: Models

See also [Writing your first Django app, part 2](https://docs.djangoproject.com/en/5.0/intro/tutorial02/).

## Database setup

Database settings in `settings.py`:

In [ ]:
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': BASE_DIR / 'db.sqlite3',
    }
}

Run initial migrations:

## Creating a simple model

Create model `Post` in `models.py`:

In [ ]:
from django.db import models

# Create your models here.

class Post(models.Model):
    title = models.CharField(max_length=100)
    text = models.TextField(max_length=1000)

Register model in `admin.py`:

In [ ]:
from django.contrib import admin

from .models import Post

# Register your models here.

admin.site.register(Post)

Create and execute migrations.

Explore admin interface at http://localhost:8000/admin

## Creating relationships

Create model `Category` in `models.py` **above** the `Post` model:

In [ ]:
from django.db import models

# Create your models here.

class Category(models.Model):
    title = models.CharField(max_length=100)

# ...

Register model in `admin.py`:

In [ ]:
from django.contrib import admin

from .models import Category, Post

# Register your models here.

admin.site.register(Category) # NEW
admin.site.register(Post)

Add relation `category` to model `Post`.

In [ ]:
# ...

class Post(models.Model):
    title = models.CharField(max_length=100)
    text = models.TextField(max_length=1000)
    category = models.ForeignKey(Category, on_delete=models.SET_NULL)

# ...

Create and execute migrations: